In [1]:
#Installs
import sys

!pip install ssh-pymongo
!pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 24.2 MB/s eta 0:00:00
  Created wheel for ssh-pymongo: filename=ssh_pymongo-1.0.4-py3-none-any.whl size=2566 sha256=f6438be71f5bf38f48f2df2697780f23bbff2910683a08fdfd922b6edafa2351
  Stored in directory: /root/.cache/pip/wheels/1d/6a/9d/dee25ecdecdb43b31f2494250adceca24ac30f0f3993113be6
Successfully built ssh-pymongo
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from pymongo import UpdateOne
from ssh_pymongo import MongoSession
import pprint

In [3]:
#Importing essential tools
import os
import re
import pandas as pd
import numpy as np
import gensim
import gensim.corpora as corpora
import nltk
import pickle 

import seaborn as sns;

from wordcloud import WordCloud
from gensim.utils import simple_preprocess
from gensim.models.coherencemodel import CoherenceModel
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 62.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatibl

In [6]:
import warnings
#warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore")

In [7]:
#Set host and remotely connect to mongodb
MONGO_HOST = "ip_address"
MONGO_USER = "username"
MONGO_PASS = "password"
MONGO_PORT = 443

session = MongoSession(MONGO_HOST,
    port=MONGO_PORT,
    user=MONGO_USER,
    password=MONGO_PASS,
    uri='mongodb:/'+MONGO_HOST+':'+str(MONGO_PORT)
)

In [8]:
clean = session.connection['tidy']

In [9]:
def find_best_num_topics(num_topics, html_corpus, html_dict, df):
  coherence = np.zeros_like(num_topics, dtype=float)
  for num_topic in num_topics:
    print('Checking coherence for ' + str(num_topic) + ' topics...')
    lda = gensim.models.LdaMulticore(corpus=html_corpus, id2word=html_dict, num_topics=num_topic, random_state=1)

    coherence_model = CoherenceModel(model=lda, texts=df['tokens'], 
                                    dictionary=html_dict, coherence='c_v')
    coherence_lda_model_base = coherence_model.get_coherence()
    coherence[num_topic - num_topics[0]] = coherence_lda_model_base

  return (np.argmax(coherence)+num_topics[0], np.max(coherence))

In [10]:
def get_topics(lda, best_topic_num):
  keywords_set = set()

  topn_title_words = {'Topic_' + str(i): [word for word, prob in lda.show_topic(i, topn=best_topic_num)] for i in range(0, lda.num_topics)}

  for x, y in topn_title_words.items():
    for i in y:
      keywords_set.add(i)

  return list(keywords_set)

In [11]:
def get_to_insert_list(keywords):
  to_insert = []
  curWord = 1
  totWord = len(keywords)
  for keyword in keywords:
    print('----------')
    print('Currently modeling for keyword: ' + keyword + ' (' + str(curWord) + ' of ' + str(totWord) + ')')
    print('----------')
    curWord += 1
    docs_to_model = db_clean.find({'keyword' : keyword}, {'Award' : 1, 'tokens' : 1})
    df = pd.DataFrame(list(docs_to_model))
    html_text = df['tokens'].to_list()

    #Prepare data for LDA input

    # Create Dictionary
    html_dict = corpora.Dictionary(html_text)

    # Term Document Frequency
    html_corpus = [html_dict.doc2bow(text) for text in html_text]

    num_topics = np.arange(3,11, dtype=int)
    best_topic_num, coherence = find_best_num_topics(num_topics, html_corpus, html_dict, df)
    lda = gensim.models.LdaMulticore(corpus=html_corpus, id2word=html_dict, num_topics=best_topic_num)

    perplexity = lda.log_perplexity(html_corpus)

    topics = get_topics(lda, best_topic_num)

    to_insert.append({'keyword' : keyword, 'Award' : df['Award'][0], 'topics' : topics, 'coherence' : coherence, 'perplexity' : perplexity})

  return to_insert

In [14]:
#'DIBBSAward2023-04-13', 'DIBBSAward2023-04-15', 'CICIAward2023-04-10', 'CSSIAward2023-04-16', 'MRIAward2023-04-16', 'CCFAward2023-04-11', 'SI2Award2023-04-24'
#still need 'CCFAward2023-04-11', 

collection_names = ['CCFAward2023-04-11']

In [15]:
topic_collection = session.connection['keywords']

for cur_collection in collection_names:
  db_clean = clean[cur_collection]
  keywords = db_clean.distinct('keyword')
  print('Currently modeling: ' + cur_collection + ' ('+str(len(keywords))+' keywords)')
  print(keywords)
  print('-------------------')

  to_insert = get_to_insert_list(keywords)

  db_topics = topic_collection[cur_collection]

  print('Inserting into db...\n')
  db_topics.insert_many(to_insert)

Currently modeling: CCFAward2023-04-11 (59 keywords)
['af', 'algorithms', 'analysis', 'applications', 'based', 'br', 'career', 'cif', 'circuits', 'collaborative', 'communication', 'computational', 'computer', 'computing', 'conference', 'data', 'design', 'develop', 'dna', 'efficient', 'energy', 'grant', 'high', 'information', 'international', 'language', 'learning', 'medium', 'methods', 'modeling', 'networks', 'new', 'nsf', 'parallel', 'performance', 'problems', 'processing', 'program', 'programming', 'project', 'quantum', 'research', 'scale', 'science', 'shf', 'signal', 'small', 'software', 'student', 'students', 'support', 'system', 'systems', 'techniques', 'testing', 'theory', 'tools', 'travel', 'using']
-------------------
----------
Currently modeling for keyword: af (1 of 59)
----------


Checking coherence for 3 topics...


Checking coherence for 4 topics...


Checking coherence for 5 topics...


Checking coherence for 6 topics...


...

Checking coherence for 7 topics...


Checking coherence for 8 topics...


Checking coherence for 9 topics...


Checking coherence for 10 topics...


Inserting into db...



# Individual Testing


In [ ]:
clean = session.connection['tidy']

In [ ]:
db_clean = clean['CSSIAward2023-04-16']

In [ ]:
keywords = db_clean.distinct('keyword')

In [ ]:
keywords

['agriculture', 'communication', 'develop', 'dibbs', 'photochemical']

In [ ]:
docs_to_model = db_clean.find({'keyword' : keywords[0]}, {'Award' : 1, 'tokens' : 1})

In [ ]:
df = pd.DataFrame(list(docs_to_model))
df.head()

,_id,Award,tokens
0,6440840da8cdca1ec8c5e6f7,CSSI Award,"[sustained, scientific, innovation, cssi, la, ..."
1,6440840da8cdca1ec8c5e6fa,CSSI Award,"[cssi, skip, content, cssi, news, awards, fell..."
2,6440840da8cdca1ec8c5e6fc,CSSI Award,"[cssi, inc, receives, major, nextgen, engineer..."
3,6440840da8cdca1ec8c5e6ff,CSSI Award,"[faculty, staff, awards, gustavson, school, bu..."
4,6440840da8cdca1ec8c5e702,CSSI Award,"[secafjoin, nowloginmy, accountsecaf, chatcont..."


In [ ]:
html_text = df['tokens'].to_list()

In [ ]:
#Prepare data for LDA input

# Create Dictionary
html_dict = corpora.Dictionary(html_text)

# Term Document Frequency
html_corpus = [html_dict.doc2bow(text) for text in html_text]

# View
print(html_corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 2), (14, 4), (15, 1), (16, 1), (17, 5), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 7), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


In [ ]:
num_topics = np.arange(1,11, dtype=int)
num_topics

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [ ]:
# number of topics
num_topics = 10

# Build LDA model
lda = gensim.models.LdaMulticore(corpus=html_corpus, id2word=html_dict, num_topics=num_topics)

In [ ]:
coherence = np.zeros_like(num_topics, dtype=float)
for num_topic in num_topics:
  lda = gensim.models.LdaMulticore(corpus=html_corpus, id2word=html_dict, num_topics=num_topic)

  coherence_model = CoherenceModel(model=lda, texts=df['tokens'], 
                                   dictionary=html_dict, coherence='c_v')
  coherence_lda_model_base = coherence_model.get_coherence()
  coherence[num_topic-1] = coherence_lda_model_base

In [ ]:
coherence

array([0.29752546, 0.29673519, 0.31893501, 0.33195138, 0.29111795,
       0.278633  , 0.30832297, 0.2958319 , 0.29526621, 0.35159779])

In [ ]:
best_topic_num = np.argmax(coherence)+1

In [ ]:
get_topics(lda, best_topic_num)

['award',
 'research',
 'logistics',
 'credit',
 'information',
 'transport',
 'engineering',
 'science',
 'proposal',
 'cssi',
 'international',
 'transportation',
 'trucking',
 'nsf',
 'apply',
 'services',
 'https',
 'university',
 'hours',
 'national',
 'de',
 'express',
 'foundation',
 'data',
 'program']

In [ ]:
# Visualize the title topics
titles_prepared = pyLDAvis.gensim.prepare(lda, html_corpus, html_dict)

pyLDAvis.display(titles_prepared)

  default_term_info = default_term_info.sort_values(



In [ ]:
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]
# Create Topics
topics = [' '.join(t[0:10]) for t in words]

In [ ]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
research nsf science trucking transport program cssi transportation logistics services

------ Topic 1 ------
research congressional science program trucking transport transportation data award engineering

------ Topic 2 ------
research science nsf program national data trucking foundation transport award

------ Topic 3 ------
research nsf program science trucking national proposal transport data support

------ Topic 4 ------
trucking transport logistics transportation research de services program express science

------ Topic 5 ------
research program nsf science proposal national data engineering award trucking

------ Topic 6 ------
research science apply program nsf award national filter proposal foundation

------ Topic 7 ------
https research science nsf program trucking services data proposal transport

------ Topic 8 ------
trucking transport transportation services logistics science nsf de company express

------ Topic 9 ------
research science nsf pro

In [ ]:
#Output list of keywords for corresponding dataset.
keywords_set = set()

topn_title_words = {'Topic_' + str(i): [word for word, prob in lda.show_topic(i, topn=best_topic_num)] for i in range(0, lda.num_topics)}

for x, y in topn_title_words.items():
  for i in y:
    keywords_set.add(i)

#print(len(keywords_set))
#print(list(keywords_set))
keywords_set

{'apply',
 'award',
 'credit',
 'cssi',
 'data',
 'de',
 'engineering',
 'express',
 'foundation',
 'hours',
 'https',
 'information',
 'international',
 'logistics',
 'national',
 'nsf',
 'program',
 'proposal',
 'research',
 'science',
 'services',
 'transport',
 'transportation',
 'trucking',
 'university'}

In [ ]:
to_insert = []
for keyword in keywords:
  docs_to_model = db_clean.find({'keyword' : keyword}, {'Award' : 1, 'tokens' : 1})
  df = pd.DataFrame(list(docs_to_model))
  html_text = df['tokens'].to_list()

  #Prepare data for LDA input

  # Create Dictionary
  html_dict = corpora.Dictionary(html_text)

  # Term Document Frequency
  html_corpus = [html_dict.doc2bow(text) for text in html_text]

  num_topics = np.arange(3,11, dtype=int)
  best_topic_num, coherence = find_best_num_topics(num_topics, html_corpus, html_dict, df)
  lda = gensim.models.LdaMulticore(corpus=html_corpus, id2word=html_dict, num_topics=best_topic_num)

  topics = get_topics(lda, best_topic_num)

  to_insert.append({'keyword' : keyword, 'Award' : df['Award'][0], 'topics' : topics, 'coherence' : coherence})

In [ ]:
to_insert

[{'keyword': 'agriculture',
  'Award': 'CSSI Award',
  'topics': ['award',
   'research',
   'logistics',
   'transport',
   'engineering',
   'science',
   'proposal',
   'cssi',
   'transportation',
   'project',
   'trucking',
   'nsf',
   'apply',
   'services',
   'filter',
   'national',
   'proposals',
   'congressional',
   'de',
   'foundation',
   'express',
   'data',
   'program',
   'school'],
  'coherence': 0.3556962012564491},
 {'keyword': 'communication',
  'Award': 'CSSI Award',
  'topics': ['inc',
   'puerto',
   'work',
   'award',
   'research',
   'science',
   'cssi',
   'rico',
   'project',
   'apply',
   'nsf',
   'filter',
   'us',
   'produce',
   'new',
   'national',
   'de',
   'data',
   'program',
   'school'],
  'coherence': 0.3289025504368195},
 {'keyword': 'develop',
  'Award': 'CSSI Award',
  'topics': ['research'],
  'coherence': 0.4030589612877047},
 {'keyword': 'dibbs',
  'Award': 'CSSI Award',
  'topics': ['nsf', 'research', 'program', 'science']

In [ ]:
base_perplexity = lda.log_perplexity(html_corpus)
print('\nPerplexity: ', base_perplexity)

# Compute Coherence Score
coherence_model = CoherenceModel(model=lda, texts=df['tokens'], 
                                   dictionary=html_dict, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -10.018319702317331

Coherence Score:  0.3461107557393711


In [ ]:
topic_collection = session.connection['keywords']
db_topics = topic_collection['CSSIAward2023-04-16']

In [ ]:
db_topics.insert_many(to_insert)

In [32]:
session.stop()